# Предобработка данных Ростуризма для собственного дашборда – Внутренний турпоток

Проблематика работы:
- Выгрузка данных от Ростуризма неудобная, содержит накопленные квартальные значения по годам, вместо значений по кварталам
- Нужно предобработать данные с отчетов Ростуризма: https://rosstat.gov.ru/statistics/turizm для построения собственного дашборда для мониторинга основных показателей индустрии


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from numpy import median


pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)



In [2]:
people_23 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='1.2023', skiprows=3)
people_24 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='1.2024', skiprows=3)
people_25 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='1.2025', skiprows=3)

nights_23 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='2.2023', skiprows=3)
nights_24 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='2.2024', skiprows=3)
nights_25 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='2.2025', skiprows=3)

income_23 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='3.2023', skiprows=3)
income_24 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='3.2024', skiprows=3)
income_25 = pd.read_excel ('Дашборд по туризму/KSR_mes_sub_05-2025.xlsx', sheet_name='3.2025', skiprows=3)

In [3]:
people_list = [people_23,people_24,people_25]
nights_list = [nights_23,nights_24,nights_25]
income_list = [income_23,income_24,income_25]

In [4]:
data = pd.DataFrame(columns=['date','region','people','nights','income'])

In [5]:
data

,date,region,people,nights,income


In [6]:
income_24

,Unnamed: 0,январь 2024 г.,январь-февраль 2024 г.,январь-март 2024 г.,январь-апрель 2024 г.,январь-май 2024 г.,январь-июнь 2024 г.,январь-июль 2024 г.,январь-август 2024 г.,январь-сентябрь 2024 г.,январь-октябрь 2024 г.,январь-ноябрь 2024 г.,январь-декабрь 2024 г.
0,Российская Федерация,57142375.60,114624495.80,180655687.80,248742606.50,320804635.80,433960573.30,574509974.30,716884995.80,821038686.80,904047275.00,974629716.00,1042236286.10
1,Центральный федеральный округ,19471778.00,38879106.70,57822441.80,77373177.90,96926372.20,123804639.60,159239837.30,195856469.40,224024462.20,247099033.50,268495264.90,292555797.60
2,Белгородская область,110392.80,252234.40,444691.20,611518.70,763086.00,942942.10,1128964.10,1290339.80,1447740.10,1607185.60,1774345.00,1989816.30
3,Брянская область,59762.90,119832.10,209492.10,316256.80,421856.90,705269.60,1056480.10,1293838.40,1393445.50,1555595.80,1696277.70,1809963.50
4,Владимирская область,855827.70,1480710.30,2133586.00,2609216.10,3102714.10,3951438.60,4780024.40,5579352.10,6007604.90,6480929.70,6924010.00,7358116.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Сахалинская область,197416.40,463061.30,751787.50,961120.40,1178930.10,1434061.60,1801166.50,2206582.20,2621366.30,2938975.80,3198961.60,3436514.60
94,Еврейская автономная область,18489.30,41973.80,73599.80,102493.40,131634.40,174947.40,210902.40,241131.70,266733.50,307569.20,332293.00,347845.10
95,Чукотский автономный округ,17776.30,43405.30,75421.90,103448.20,131949.30,153765.10,177851.80,201451.70,229850.50,249809.50,271154.80,285648.20
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
a=2015
pd.to_datetime('01-03-'+str(a))

Timestamp('2015-01-03 00:00:00')

In [15]:
for i in range(len(people_list)):
    a=2023+i
    display(a)
    people = people_list[i]
    night = nights_list[i]
    income = income_list[i]
    
    people.columns = people.columns.str.replace(' 2023 г.1','')
    night.columns = night.columns.str.replace(' 2023 г.1','')
    income.columns = income.columns.str.replace(' 2023 г.1','')

    people.columns = people.columns.str.replace(' 2023 г.','')
    night.columns = night.columns.str.replace(' 2023 г.','')
    income.columns = income.columns.str.replace(' 2023 г.','')

    people.columns = people.columns.str.replace(' 2024 г.1','')
    night.columns = night.columns.str.replace(' 2024 г.1','')
    income.columns = income.columns.str.replace(' 2024 г.1','')

    people.columns = people.columns.str.replace(' 2024 г.','')
    night.columns = night.columns.str.replace(' 2024 г.','')
    income.columns = income.columns.str.replace(' 2024 г.','')

    people.columns = people.columns.str.replace(' 2025 г.1','')
    night.columns = night.columns.str.replace(' 2025 г.1','')
    income.columns = income.columns.str.replace(' 2025 г.1','')

    people.columns = people.columns.str.replace(' 2025 г.','')
    night.columns = night.columns.str.replace(' 2025 г.','')
    income.columns = income.columns.str.replace(' 2025 г.','')

    people['декабрь_л'] = (people['январь-декабрь'] - people['январь-ноябрь'])
    people['ноябрь_л'] = (people['январь-ноябрь'] - people['январь-октябрь'])
    people['октябрь_л'] = (people['январь-октябрь'] - people['январь-сентябрь'])
    people['сентябрь_л'] = (people['январь-сентябрь'] - people['январь-август'])
    people['август_л'] = (people['январь-август'] - people['январь-июль'])
    people['июль_л'] = (people['январь-июль'] - people['январь-июнь'])
    people['июнь_л'] = (people['январь-июнь'] - people['январь-май'])
    people['май_л'] = (people['январь-май'] - people['январь-апрель'])
    people['апрель_л'] = (people['январь-апрель'] - people['январь-март'])
    people['март_л'] = (people['январь-март'] - people['январь-февраль'])
    people['февраль_л'] = (people['январь-февраль'] - people['январь'])
    people['январь_л'] = people['январь']

    people = people[['Unnamed: 0','январь_л','февраль_л','март_л','апрель_л','май_л','июнь_л','июль_л','август_л','сентябрь_л','октябрь_л','ноябрь_л','декабрь_л']]

    night['декабрь_н'] = (night['январь-декабрь'] - night['январь-ноябрь'])
    night['ноябрь_н'] = (night['январь-ноябрь'] - night['январь-октябрь'])
    night['октябрь_н'] = (night['январь-октябрь'] - night['январь-сентябрь'])
    night['сентябрь_н'] = (night['январь-сентябрь'] - night['январь-август'])
    night['август_н'] = (night['январь-август'] - night['январь-июль'])
    night['июль_н'] = (night['январь-июль'] - night['январь-июнь'])
    night['июнь_н'] = (night['январь-июнь'] - night['январь-май'])
    night['май_н'] = (night['январь-май'] - night['январь-апрель'])
    night['апрель_н'] = (night['январь-апрель'] - night['январь-март'])
    night['март_н'] = (night['январь-март'] - night['январь-февраль'])
    night['февраль_н'] = (night['январь-февраль'] - night['январь'])
    night['январь_н'] = night['январь']

    night = night[['Unnamed: 0','январь_н','февраль_н','март_н','апрель_н','май_н','июнь_н','июль_н','август_н','сентябрь_н','октябрь_н','ноябрь_н','декабрь_н']]

    income['декабрь_д'] = (income['январь-декабрь'] - income['январь-ноябрь'])*1000
    income['ноябрь_д'] = (income['январь-ноябрь'] - income['январь-октябрь'])*1000
    income['октябрь_д'] = (income['январь-октябрь'] - income['январь-сентябрь'])*1000
    income['сентябрь_д'] = (income['январь-сентябрь'] - income['январь-август'])*1000
    income['август_д'] = (income['январь-август'] - income['январь-июль'])*1000
    income['июль_д'] = (income['январь-июль'] - income['январь-июнь'])*1000
    income['июнь_д'] = (income['январь-июнь'] - income['январь-май'])*1000
    income['май_д'] = (income['январь-май'] - income['январь-апрель'])*1000
    income['апрель_д'] = (income['январь-апрель'] - income['январь-март'])*1000
    income['март_д'] = (income['январь-март'] - income['январь-февраль'])*1000
    income['февраль_д'] = (income['январь-февраль'] - income['январь'])*1000
    income['январь_д'] = income['январь']*1000

    income = income[['Unnamed: 0','январь_д','февраль_д','март_д','апрель_д','май_д','июнь_д','июль_д','август_д','сентябрь_д','октябрь_д','ноябрь_д','декабрь_д']]
    
#    print (a, 'people')
#    display(people)
#    print (a, 'night')
#    display(night)
#    print (a, 'income')
#    display(income)


    
    
    month = people.merge(night, how='outer', on='Unnamed: 0').merge(income, how='outer', on='Unnamed: 0')



    print (a, 'year')
    display(month)

    m1= month[['Unnamed: 0','январь_л','январь_н','январь_д']].rename(columns={'Unnamed: 0':'region','январь_л':'people','январь_н':'nights','январь_д':'income'})
    m1['date'] = pd.to_datetime('30-01-'+str(a),dayfirst=True)
#    print (a, 'm1')
#    display(m1)

    m2= month[['Unnamed: 0','февраль_л','февраль_н','февраль_д']].rename(columns={'Unnamed: 0':'region','февраль_л':'people','февраль_н':'nights','февраль_д':'income'})
    m2['date'] = pd.to_datetime('28-02-'+str(a),dayfirst=True)
#    print (a, 'm2')
#    display(m2)

    m3= month[['Unnamed: 0','март_л','март_н','март_д']].rename(columns={'Unnamed: 0':'region','март_л':'people','март_н':'nights','март_д':'income'})
    m3['date'] = pd.to_datetime('30-03-'+str(a),dayfirst=True)
#    print (a, 'm3')
#    display(m3)

    m4= month[['Unnamed: 0','апрель_л','апрель_н','апрель_д']].rename(columns={'Unnamed: 0':'region','апрель_л':'people','апрель_н':'nights','апрель_д':'income'})
    m4['date'] = pd.to_datetime('30-04-'+str(a),dayfirst=True)
#    print (a, 'm4')
#    display(m4)

    m5= month[['Unnamed: 0','май_л','май_н','май_д']].rename(columns={'Unnamed: 0':'region','май_л':'people','май_н':'nights','май_д':'income'})
    m5['date'] = pd.to_datetime('30-05-'+str(a),dayfirst=True)
#    print (a, 'm5')
#    display(m5)

    m6= month[['Unnamed: 0','июнь_л','июнь_н','июнь_д']].rename(columns={'Unnamed: 0':'region','июнь_л':'people','июнь_н':'nights','июнь_д':'income'})
    m6['date'] = pd.to_datetime('30-06-'+str(a),dayfirst=True)
#    print (a, 'm6')
#    display(m6)

    m7= month[['Unnamed: 0','июль_л','июль_н','июль_д']].rename(columns={'Unnamed: 0':'region','июль_л':'people','июль_н':'nights','июль_д':'income'})
    m7['date'] = pd.to_datetime('30-07-'+str(a),dayfirst=True)
#    print (a, 'm7')
#    display(m7)

    m8= month[['Unnamed: 0','август_л','август_н','август_д']].rename(columns={'Unnamed: 0':'region','август_л':'people','август_н':'nights','август_д':'income'})
    m8['date'] = pd.to_datetime('30-08-'+str(a),dayfirst=True)
#    print (a, 'm8')
#    display(m8)

    m9= month[['Unnamed: 0','сентябрь_л','сентябрь_н','сентябрь_д']].rename(columns={'Unnamed: 0':'region','сентябрь_л':'people','сентябрь_н':'nights','сентябрь_д':'income'})
    m9['date'] = pd.to_datetime('30-09-'+str(a),dayfirst=True)
#    print (a, 'm9')
#    display(m9)

    m10= month[['Unnamed: 0','октябрь_л','октябрь_н','октябрь_д']].rename(columns={'Unnamed: 0':'region','октябрь_л':'people','октябрь_н':'nights','октябрь_д':'income'})
    m10['date'] = pd.to_datetime('30-10-'+str(a),dayfirst=True)
#    print (a, 'm10')
#    display(m10)

    m11= month[['Unnamed: 0','ноябрь_л','ноябрь_н','ноябрь_д']].rename(columns={'Unnamed: 0':'region','ноябрь_л':'people','ноябрь_н':'nights','ноябрь_д':'income'})
    m11['date'] = pd.to_datetime('30-11-'+str(a),dayfirst=True)
#    print (a, 'm11')
#    display(m11)

    m12= month[['Unnamed: 0','декабрь_л','декабрь_н','декабрь_д']].rename(columns={'Unnamed: 0':'region','декабрь_л':'people','декабрь_н':'nights','декабрь_д':'income'})
    m12['date'] = pd.to_datetime('30-12-'+str(a),dayfirst=True)
#    print (a, 'm12')
#    display(m12)

    data = pd.concat([data,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12])

2023

2023 year


,Unnamed: 0,январь_л,февраль_л,март_л,апрель_л,май_л,июнь_л,июль_л,август_л,сентябрь_л,октябрь_л,ноябрь_л,декабрь_л,январь_н,февраль_н,март_н,апрель_н,май_н,июнь_н,июль_н,август_н,сентябрь_н,октябрь_н,ноябрь_н,декабрь_н,январь_д,февраль_д,март_д,апрель_д,май_д,июнь_д,июль_д,август_д,сентябрь_д,октябрь_д,ноябрь_д,декабрь_д
0,1 Без учета статистической информации по Донец...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Алтайский край,45269.00,51779.00,60245.00,65008.00,71878.00,90035.00,111149.00,87933.00,63086.00,62393.00,54488.00,51403.00,179750.00,205621.00,240836.00,242128.00,286236.00,435263.00,524412.00,401636.00,269839.00,282220.00,252467.00,210906.00,956024600.00,765567400.00,922024600.00,1083843700.00,1165424900.00,1408121300.00,1841798300.00,1688240100.00,1335672900.00,1226972400.00,1037485000.00,760263400.00
2,Амурская область,16021.00,18071.00,17896.00,18558.00,27153.00,18449.00,18719.00,24929.00,23499.00,28378.00,21295.00,19768.00,36933.00,55118.00,51431.00,77569.00,66492.00,75479.00,85355.00,99008.00,71034.00,108211.00,84183.00,88707.00,69653700.00,100161500.00,126677400.00,157785200.00,136074800.00,164695100.00,156724600.00,181483100.00,125109100.00,179276100.00,184350200.00,231385100.00
3,Архангельская область,25113.00,27302.00,35655.00,26222.00,24380.00,32717.00,41472.00,36884.00,26324.00,28937.00,24006.00,29397.00,73285.00,96785.00,129468.00,96799.00,85486.00,120674.00,165761.00,126727.00,84514.00,105146.00,96172.00,95798.00,93621200.00,162216700.00,213906300.00,193020400.00,146961000.00,209718800.00,196772700.00,227083400.00,145973900.00,207066200.00,211972200.00,202386200.00
4,Архангельская область (кроме Ненецкого автоном...,24738.00,26606.00,33995.00,25044.00,23717.00,30729.00,41417.00,35841.00,25665.00,28018.00,23481.00,28085.00,72012.00,94175.00,123102.00,94885.00,84094.00,113453.00,165707.00,120080.00,82280.00,101344.00,94239.00,91932.00,89752800.00,157608900.00,196761600.00,186684800.00,143171500.00,191896100.00,196545800.00,199170600.00,138371600.00,192058300.00,204309700.00,187008400.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Ярославская область,62077.00,78793.00,64322.00,80567.00,83474.00,115109.00,138039.00,128574.00,94392.00,84357.00,91031.00,85801.00,135724.00,164977.00,160548.00,168323.00,181512.00,369705.00,409972.00,361992.00,169924.00,171530.00,170802.00,154803.00,370955500.00,358594000.00,366921700.00,406213700.00,414488700.00,756536000.00,933785700.00,900600300.00,523807200.00,484978200.00,477601700.00,466178600.00
94,г.Москва,830390.00,783877.00,892243.00,895860.00,921677.00,975026.00,970383.00,1017377.00,956761.00,993285.00,994554.00,1029972.00,3252734.00,3119254.00,3616182.00,3618519.00,3663275.00,3842587.00,3917584.00,4300549.00,4013886.00,4278779.00,4298430.00,4383616.00,6889951100.00,7315747500.00,8515712300.00,8450944500.00,8181004900.00,8723718000.00,11300950600.00,13284122600.00,13808074300.00,10433772200.00,11169828400.00,13386803200.00
95,г.Санкт-Петербург,292949.00,285040.00,352794.00,381535.00,432257.00,521783.00,531106.00,579313.00,478750.00,469133.00,431539.00,415016.00,903818.00,875932.00,1099392.00,1177327.00,1348514.00,1717723.00,1843335.00,1803428.00,1399511.00,1346700.00,1284419.00,1142911.00,2580502200.00,2458629300.00,2866551800.00,3054716000.00,4189138700.00,9251267300.00,8294638100.00,7243627200.00,5975183600.00,4012788100.00,3967566000.00,4339152700.00
96,г.Севастополь,7546.00,4611.00,5160.00,6663.00,11060.00,31117.00,54854.00,40201.00,28314.00,6531.00,2835.00,3238.00,49661.00,33772.00,36020.00,35674.00,38311.00,148531.00,227170.00,281655.00,109246.00,46492.00,19685.00,19265.00,38811800.00,36198500.00,83846600.00,113950200.00,183848200.00,231743700.00,198213500.00,329893300.00,180810000.00,87460300.00,17607300.00,23488300.00


2024

2024 year


,Unnamed: 0,январь_л,февраль_л,март_л,апрель_л,май_л,июнь_л,июль_л,август_л,сентябрь_л,октябрь_л,ноябрь_л,декабрь_л,январь_н,февраль_н,март_н,апрель_н,май_н,июнь_н,июль_н,август_н,сентябрь_н,октябрь_н,ноябрь_н,декабрь_н,январь_д,февраль_д,март_д,апрель_д,май_д,июнь_д,июль_д,август_д,сентябрь_д,октябрь_д,ноябрь_д,декабрь_д
0,1 Без учета статистической информации по Донец...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Алтайский край,53717.00,57488.00,64208.00,70255.00,77248.00,110097.00,129045.00,107294.00,67254.00,68459.00,59143.00,58709.00,192227.00,225917.00,260082.00,256788.00,279025.00,465250.00,563602.00,458582.00,278249.00,286431.00,258076.00,213063.00,901363300.00,820060700.00,1037320200.00,1059780000.00,1145845800.00,1541066200.00,2034097700.00,1952087500.00,1549740700.00,1343594200.00,1085248400.00,913089500.00
2,Амурская область,18602.00,16561.00,18344.00,23783.00,20704.00,19873.00,23947.00,29029.00,26262.00,27534.00,22767.00,25739.00,55635.00,60712.00,76338.00,92793.00,82999.00,88032.00,108937.00,115857.00,93093.00,133841.00,108158.00,96835.00,93542300.00,112441500.00,137085300.00,197697600.00,163455700.00,187003700.00,192491600.00,212708100.00,150848300.00,253166700.00,231899200.00,347862600.00
3,Архангельская область,26985.00,31330.00,31312.00,27340.00,28647.00,36975.00,44130.00,35106.00,35065.00,29273.00,25122.00,24940.00,82056.00,93877.00,102102.00,94387.00,89778.00,126706.00,148589.00,117143.00,85690.00,94014.00,91007.00,79585.00,130264300.00,181217900.00,197678800.00,220926400.00,202291300.00,249891100.00,297575000.00,253884600.00,204771300.00,228602900.00,229130000.00,199624800.00
4,Архангельская область (кроме Ненецкого автоном...,25756.00,30752.00,30300.00,27169.00,28133.00,34661.00,43691.00,34740.00,33747.00,28630.00,24640.00,24458.00,78019.00,93204.00,100756.00,93466.00,88397.00,123713.00,146997.00,116482.00,84038.00,92842.00,88909.00,77549.00,116554700.00,178792900.00,192503200.00,216826600.00,196141600.00,242018300.00,291224000.00,251256400.00,198696000.00,223355900.00,220625300.00,190026000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Ярославская область,82100.00,78412.00,89795.00,88819.00,109715.00,114669.00,143558.00,126908.00,75084.00,86496.00,69009.00,74849.00,150750.00,146424.00,172519.00,195040.00,184068.00,360361.00,444701.00,372931.00,149903.00,183804.00,167541.00,147257.00,537367900.00,465955700.00,581734100.00,544488500.00,571105600.00,941812300.00,1223558600.00,1147694500.00,629743000.00,616583100.00,503705800.00,566958500.00
94,г.Москва,1014108.00,973612.00,985204.00,951354.00,968566.00,1040043.00,1088087.00,1101913.00,958087.00,1010924.00,972096.00,1049911.00,4486156.00,4303012.00,4451600.00,4457120.00,4496095.00,4749865.00,4985511.00,5120212.00,4488041.00,4933772.00,4787630.00,4990435.00,10536121800.00,11145663600.00,10634027900.00,11381140900.00,11310093700.00,12501060900.00,18356810500.00,20268215100.00,18378684500.00,13348207400.00,12507117000.00,15068964600.00
95,г.Санкт-Петербург,396194.00,366467.00,439988.00,487833.00,530301.00,615704.00,661079.00,631813.00,511283.00,632052.00,566274.00,532583.00,1325155.00,1203761.00,1500637.00,1612813.00,1786765.00,2177374.00,2443324.00,2335781.00,1722270.00,2011080.00,1762814.00,1583463.00,3750445800.00,3233836300.00,4111746400.00,4568798700.00,5661182900.00,12049468200.00,10697704400.00,9416763200.00,7015405500.00,6026689100.00,5083512000.00,5631373300.00
96,г.Севастополь,2907.00,3061.00,4357.00,4833.00,8324.00,23010.00,22133.00,36104.00,17779.00,7896.00,3118.00,2538.00,11658.00,10764.00,13999.00,20028.00,31014.00,143951.00,191569.00,220574.00,77928.00,38178.00,14647.00,13448.00,38105800.00,33770100.00,71054300.00,103544000.00,194683000.00,434846500.00,209456800.00,260997400.00,134397500.00,175849400.00,47504600.00,40572700.00


2025

2025 year


,Unnamed: 0,январь_л,февраль_л,март_л,апрель_л,май_л,июнь_л,июль_л,август_л,сентябрь_л,октябрь_л,ноябрь_л,декабрь_л,январь_н,февраль_н,март_н,апрель_н,май_н,июнь_н,июль_н,август_н,сентябрь_н,октябрь_н,ноябрь_н,декабрь_н,январь_д,февраль_д,март_д,апрель_д,май_д,июнь_д,июль_д,август_д,сентябрь_д,октябрь_д,ноябрь_д,декабрь_д
0,1 Без учета статистической информации по Донец...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Алтайский край,60200.00,65116.00,71876.00,64884.00,80817.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200689.00,220193.00,274963.00,260234.00,282879.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1024221100.00,957206000.00,1172348000.00,1179232100.00,1341027500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Амурская область,22130.00,25617.00,24250.00,25435.00,25996.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49362.00,72768.00,71811.00,79369.00,75176.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98490900.00,170129700.00,192022700.00,222967200.00,189350200.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Архангельская область,27855.00,27857.00,28710.00,23349.00,25826.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78723.00,82760.00,88265.00,83237.00,74316.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148670900.00,205779800.00,219306200.00,210884600.00,207397600.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Архангельская область (кроме Ненецкого автоном...,27004.00,26156.00,27115.00,23003.00,24789.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77498.00,80146.00,85488.00,82593.00,73027.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143414400.00,196813800.00,207778600.00,207808700.00,201245800.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Ярославская область,85486.00,75066.00,78279.00,81941.00,104598.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163377.00,149474.00,168110.00,170441.00,186189.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,666300400.00,489807300.00,558317000.00,537716000.00,677317100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,г.Москва,959151.00,884607.00,981902.00,1010697.00,1060958.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4697253.00,4315358.00,5007363.00,5253730.00,5454940.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11210518900.00,11156035700.00,11478579700.00,13307008200.00,13169094300.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,г.Санкт-Петербург,455674.00,414444.00,501526.00,575666.00,652897.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1492671.00,1411271.00,1763261.00,1890431.00,2132713.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4526215700.00,3911490400.00,4473300600.00,5393945300.00,7782426700.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,г.Севастополь,3618.00,2689.00,4967.00,6850.00,9449.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8914.00,9834.00,14443.00,21223.00,30099.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80286400.00,38852000.00,91441800.00,144344900.00,161301100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data


,date,region,people,nights,income
0,2023-01-30,1 Без учета статистической информации по Донец...,NaN,NaN,NaN
1,2023-01-30,Алтайский край,45269.00,179750.00,956024600.00
2,2023-01-30,Амурская область,16021.00,36933.00,69653700.00
3,2023-01-30,Архангельская область,25113.00,73285.00,93621200.00
4,2023-01-30,Архангельская область (кроме Ненецкого автоном...,24738.00,72012.00,89752800.00
...,...,...,...,...,...
93,2025-12-30,Ярославская область,NaN,NaN,NaN
94,2025-12-30,г.Москва,NaN,NaN,NaN
95,2025-12-30,г.Санкт-Петербург,NaN,NaN,NaN
96,2025-12-30,г.Севастополь,NaN,NaN,NaN


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3528 entries, 0 to 97
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3528 non-null   datetime64[ns]
 1   region  3492 non-null   object        
 2   people  2784 non-null   float64       
 3   nights  2784 non-null   float64       
 4   income  2784 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 165.4+ KB


In [11]:
data['region'].value_counts()

region
1 Без учета статистической информации по Донецкой Народной Республике (ДНР), Луганской Народной Республике (ЛНР), Запорожской и Херсонской областям.    36
Республика Бурятия                                                                                                                                      36
Северо-Кавказский федеральный округ                                                                                                                     36
Северо-Западный федеральный округ                                                                                                                       36
Свердловская область                                                                                                                                    36
                                                                                                                                                        ..
Липецкая область                                               

In [12]:
data = data[data['region'] != '1 Без учета статистической информации по Донецкой Народной Республике (ДНР), Луганской Народной Республике (ЛНР), Запорожской и Херсонской областям.']

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3492 entries, 1 to 97
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3492 non-null   datetime64[ns]
 1   region  3456 non-null   object        
 2   people  2784 non-null   float64       
 3   nights  2784 non-null   float64       
 4   income  2784 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 163.7+ KB


In [14]:
data.to_excel('Дашборд по туризму/tourism_KSR_month.xlsx', index=False)